### Computational Economics
Alexander Skorobogatov

# Topic 2.2. Stationary distribution: another example
Political regime types:
1. Monarch. Chief of state is a monarch (either hereditary or elective) or a regent functioning on a monarch's behalf.
2. President. Chief of state is a president who may function as a chief executive or merely as titular head of state, in which case he will possess little effective power. The presiding officer of a legislative assembly or state council may qualify for the coding, even though the formal title may be that of "chairman".
3. Military. A situation in which a member of the nation's armed forces is recognized as the formal head of government.
4. Other. This category is generally used when no distinct head of state can be identified; it also includes individuals not included in (1-3), such as theocratic rulers, as well as nonmilitary individuals serving in a collegial capacity.

In [192]:
import pandas as pd
import numpy as np
# Download a dataset
url = 'https://github.com/askorobogat/CompEcon/raw/main/hist_data.xls'

In [193]:
df = pd.read_excel(url)
df.shape

(16326, 195)

In [194]:
# Look at variables
c = np.array(df.columns)
c

array(['code', 'Wbcode', 'country', 'year', 'area1', 'area2', 'area3',
       'computer1', 'computer2', 'computer3', 'computer4', 'computer5',
       'computer6', 'delta01', 'delta02', 'delta03', 'delta04', 'delta05',
       'delta06', 'delta07', 'delta08', 'delta09', 'delta10', 'delta11',
       'delta12', 'delta13', 'delta14', 'delta15', 'delta16', 'delta17',
       'delta18', 'delta19', 'delta20', 'delta21', 'delta22', 'delta23',
       'delta24', 'delta25', 'delta26', 'delta27', 'delta28', 'delta29',
       'delta30', 'delta31', 'delta32', 'delta33', 'delta34', 'delta35',
       'domestic1', 'domestic2', 'domestic3', 'domestic4', 'domestic5',
       'domestic6', 'domestic7', 'domestic8', 'domestic9', 'economics1',
       'economics2', 'economics3', 'economics4', 'economics5',
       'economics6', 'economics7', 'electoral1', 'electoral2',
       'electoral3', 'electoral4', 'electoral5', 'energy1', 'energy2',
       'energy3', 'energy4', 'indprod1', 'indprod2', 'indprod3',
       'in

In [195]:
df = df.dropna(axis='index', how='any', subset=['polit05'])
df = df[['code','country','year','polit05']]
df = w.sort_values(by=['code','year'])
df.astype({'polit05': 'int32'})

,code,country,year,polit05
33,10,Afghanistan,1919,1
34,10,Afghanistan,1920,1
35,10,Afghanistan,1921,1
36,10,Afghanistan,1922,1
37,10,Afghanistan,1923,1
...,...,...,...,...
16281,1300,Zambia,2002,2
16282,1300,Zambia,2003,2
16283,1300,Zambia,2004,2
16284,1300,Zambia,2005,2


In [196]:
df['trans'] = df.polit05.shift(-1).where(df.code.eq(df.code.shift(-1)))
df

,code,country,year,polit05,trans
33,10,Afghanistan,1919,1.0,1.0
34,10,Afghanistan,1920,1.0,1.0
35,10,Afghanistan,1921,1.0,1.0
36,10,Afghanistan,1922,1.0,1.0
37,10,Afghanistan,1923,1.0,1.0
...,...,...,...,...,...
16281,1300,Zambia,2002,2.0,2.0
16282,1300,Zambia,2003,2.0,2.0
16283,1300,Zambia,2004,2.0,2.0
16284,1300,Zambia,2005,2.0,2.0


In [197]:
tr = pd.crosstab(df.polit05, df.trans)
tr

trans,1.0,2.0,3.0,4.0
polit05,,,,
1.0,5334,46,5,9
2.0,12,8172,73,35
3.0,2,70,315,8
4.0,6,54,3,423


In [198]:
df[df['year'] == df['year'].unique()[0]]['polit05'].unique()

array([1., 2., 4.])

In [199]:
df[df['year'] == df['year'].unique()[0]]['polit05'].value_counts()

2.0    31
1.0    26
4.0     4
Name: polit05, dtype: int64

In [200]:
N = 4
M = 4
m = []
for i in range(N):
    m.append([0]*M)
for i in range(1,5):
    for j in range(1,5):
        m[i-1][j-1] = tr.loc[i,j]/tr.loc[i].sum()
m = np.array(m)
m

array([[9.88876529e-01, 8.52799407e-03, 9.26955877e-04, 1.66852058e-03],
       [1.44717800e-03, 9.85528220e-01, 8.80366618e-03, 4.22093584e-03],
       [5.06329114e-03, 1.77215190e-01, 7.97468354e-01, 2.02531646e-02],
       [1.23456790e-02, 1.11111111e-01, 6.17283951e-03, 8.70370370e-01]])

In [201]:
np.round(m,3)

array([[0.989, 0.009, 0.001, 0.002],
       [0.001, 0.986, 0.009, 0.004],
       [0.005, 0.177, 0.797, 0.02 ],
       [0.012, 0.111, 0.006, 0.87 ]])

Stationary distribution

In [202]:
def stationary_linalg(P,psi0=[None,]):
    '''Computes stationary distribution for the Markov chain given by transition probability
    matrix P. Method: linear solver.
    '''
    if psi0[0]==None:
        # degenrate initial distribution
        psi0 = [0,]*P.shape[0]
        psi0[0]=1.0
    P,psi0 = np.asarray(P),np.asarray(psi0)  # convert to np arrays (in case lists were passed)
    assert np.all(np.abs(P.sum(axis=1)-1)<1e-10), 'Passed P is not stochastic matrix'
    assert np.abs(psi0.sum()-1)<1e-10, 'Passed probabilities do not sum up to one'
    m = P.shape[0]  # dimension of the problem
    A = np.ones((m+1,m+1))  # square matrix of ones
    A[:-1,:-1] = np.eye(m)-P.T
    b = np.ones(m+1)
    b[-1]=2
    res = np.linalg.solve(A,b)
    return res[:-1]

print(stationary_linalg(m))

[0.15382483 0.77780479 0.03551511 0.03285528]


Check forecasts from the past

In [203]:
df_d = pd.get_dummies(df, columns=['polit05'])
df_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14793 entries, 33 to 16285
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   code         14793 non-null  int64  
 1   country      14793 non-null  object 
 2   year         14793 non-null  int64  
 3   trans        14567 non-null  float64
 4   polit05_1.0  14793 non-null  uint8  
 5   polit05_2.0  14793 non-null  uint8  
 6   polit05_3.0  14793 non-null  uint8  
 7   polit05_4.0  14793 non-null  uint8  
dtypes: float64(1), int64(2), object(1), uint8(4)
memory usage: 635.6+ KB


In [204]:
df_d.head()

,code,country,year,trans,polit05_1.0,polit05_2.0,polit05_3.0,polit05_4.0
33,10,Afghanistan,1919,1.0,1,0,0,0
34,10,Afghanistan,1920,1.0,1,0,0,0
35,10,Afghanistan,1921,1.0,1,0,0,0
36,10,Afghanistan,1922,1.0,1,0,0,0
37,10,Afghanistan,1923,1.0,1,0,0,0


Initial distribution

In [205]:
dist_init = []
for i in range(1,5):
    y = str('polit05_' + str(i) + '.0')
    i = df_d[df_d['year'] == df_d['year'].unique()[0]][y].describe()[1]
    dist_init.append(i)
dist_init

[0.4262295081967213, 0.5081967213114754, 0.0, 0.06557377049180328]

In [191]:
df_d[df_d['year'] == df_d['year'].unique()[0]]

,code,country,year,trans,polit05_1.0,polit05_2.0,polit05_3.0,polit05_4.0
33,10,Afghanistan,1919,1.0,1,0,0,0
121,20,Albania,1919,1.0,1,0,0,0
470,40,Argentina,1919,2.0,0,1,0,0
620,50,Australia,1919,1.0,1,0,0,0
708,61,Austria,1919,2.0,0,0,0,1
...,...,...,...,...,...,...,...,...
15078,1210,United Kingdom,1919,1.0,1,0,0,0
15265,1220,United States,1919,2.0,0,1,0,0
15463,1240,Uruguay,1919,2.0,0,1,0,0
15771,1250,Venezuela,1919,2.0,0,1,0,0
